# Méthode de réduction de variance par _importance sampling_

- Question 1 : il suffit d'utiliser le théorème de Girsanov (Cameron Martin dans ce cas)

  $\def\P{\mathbb{P}}\def\Q{\mathbb{Q}}\def\l{\lambda}$
  $\def\E{\mathbb{E}}$

  Soit $(W_t)_{t \in [0,T]}$ un mouvement brownien standard sous la probabilité $\P$. Par le théorème de Cameron Martin, pour tout $\lambda \in \R$, il existe une probabilité $\Q^\lambda$ définie par $\frac{d\Q^\l}{d\P} = e^{-\l W_T - \frac{1}{2} \l^2 T}$ sous laquelle le processus $(W^\l_t)_{t \in [0,T]}$ un mouvement brownien standard.

  Puisque $(W^\l_t)_{t \in [0,T]}$ a même loi sous $\Q^\l$ que $(W_t)_{t \in [0,T]}$ sous $\P$, on a 
  $$\begin{align*}
  \E_\P[f((W_t)_{t \in [0,T]})] &= \E_{\Q^\l}[f((W^\l_t)_{t \in [0,T]})] \\
  & = \E_{\P}\left[f((W^\l_t)_{t \in [0,T]}) \frac{d\Q^\l}{d\P}\right] \\
  & = \E_{\P}[f((W_t + \lambda t)_{t \in [0,T]}) e^{-\l W_T - \frac{1}{2} \l^2 T}]
  \end{align*}$$
- Question 2 : Appliquer la formule de la question 1 à l'écriture de la variance sous la forme ${\mathbb E}[... ^2] - {\mathbb E}[... ]^2$
- Question 4 : il suffit de calculer la dérivée de $u$ et d'écrire que $v'(\lambda^\star)=0$.
  $$u''(\lambda) = T + \frac{\E\left[W_T^2 e^{- \lambda W_T} f^2((W_t, t \le T))\right]}{\E\left[e^{- \lambda W_T} f^2((W_t, t \le T))\right]} - \frac{\E\left[W_T e^{- \lambda W_T} f^2((W_t, t \le T))\right]^2}{\E\left[e^{- \lambda W_T} f^2((W_t, t \le T))\right]^2}$$
  Remarquons que l'inégalité de Cauchy Schwartz permet d'écrire que
  $$\begin{align*}
  &\frac{\E\left[W_T e^{- \lambda W_T} f^2((W_t, t \le T))\right]^2}{\E\left[e^{- \lambda W_T} f^2((W_t, t \le T))\right]^2} \\
  &\le \frac{\E\left[W_T^2 e^{- \lambda W_T} f^2((W_t, t \le T))\right]\E\left[e^{- \lambda W_T} f^2((W_t, t \le T))\right]}{\E\left[e^{- \lambda W_T} f^2((W_t, t \le T))\right]^2}\\
  &\le \frac{\E\left[W_T^2 e^{- \lambda W_T} f^2((W_t, t \le T))\right]}{\E\left[e^{- \lambda W_T} f^2((W_t, t \le T))\right]}.
  \end{align*}$$
  On conclut alors que $u''(\lambda) \ge T$.

In [1]:
using Random: randn!

"""
Compute one simulation of the BS model with J time steps over [0,T]

# Arguments
- `path` an array of size J + 1, containing a simulation of the model on output
- `G`: an array of size J filled with normal random variables.
- `lambda`: the linear drift to add to the Brownian motion in the BS model
"""
function bsAsset!(path::Array{Float64}, sigma, r, T, spot, J::Int, G::Array, lambda=0.)
    @assert length(path) == J + 1
    dt = T / J
    sqrt_dt = sqrt(dt)
    path[1] = spot
    for i in eachindex(path[1:end - 1])
        path[i + 1] = path[i] * exp((r - sigma * sigma / 2) * dt + sigma * (sqrt_dt * G[i] + lambda * dt))
    end
end

"""
Compute the discrete time barrier payoff

#Arguments
- `path::Array`: an array with size J+1 containing a single simulation of the BS model
"""
function barrierPayoff(path::Array{Float64}, strike, barrier)
    for spot in path
        if spot < barrier
            return 0
        end
    end
    return max(path[end] - strike, 0)
end

function finalWT(G::Array{Float64}, T, J::Int)
    @assert length(G) == J
    return sum(G) * sqrt(T / J)
end

function exp_minus(G::Array, J, T, lambda)
    return exp(-lambda * finalWT(G, T, J) - 0.5 * lambda * lambda * T)
end


function monteCarlo(sigma, r, T, spot, J, strike, barrier, nSamples, lambda = 0)
    price = 0.
    stdDev = 0.
    G = zeros(J)
    path = zeros(J + 1)
    for i=1:nSamples
        randn!(G)
        bsAsset!(path, sigma, r, T, spot, J, G, lambda)
        p = barrierPayoff(path, strike, barrier) *exp(-r * T) * exp_minus(G, J, T, lambda)
        price += p
        stdDev += p * p
    end
    price /= nSamples
    stdDev = stdDev / nSamples - price * price
    stdDev = sqrt(stdDev / nSamples)
    return price, 1.96 * stdDev
end

monteCarlo (generic function with 2 methods)

In [4]:
monteCarlo(0.2, 0.05, 2, 100, 24, 110, 80, 50000)

(11.14333092074296, 0.17284195811329184)

## Importance sampling

$\def\abs#1{|#1|}$
$\def\l{\lambda}$
$$u_n(\lambda) = \frac{\abs{\l}^2 T}{2} + \log \sum_{i=1}^n e^{-\l \cdot W^i_T } f^2((W^i_t, t \le T)).$$
$$u_n'(\lambda) = \l T + \frac{\sum_{i=1}^n -W^i_T e^{-\l \cdot W^i_T } f^2((W^i_t, t \le T))}{\sum_{i=1}^n e^{-\l \cdot W^i_T } f^2((W^i_t, t \le T))}.$$
$$u_n''(\lambda) = T + \frac{\sum_{i=1}^n (W^i_T)^2 e^{-\l \cdot W^i_T } f^2((W^i_t, t \le T))}{\sum_{i=1}^n e^{-\l \cdot W^i_T } f^2((W^i_t, t \le T))} + \frac{\left(\sum_{i=1}^n W^i_T e^{-\l \cdot W^i_T } f^2((W^i_t, t \le T))\right)^2}{\left(\sum_{i=1}^n e^{-\l \cdot W^i_T } f^2((W^i_t, t \le T))\right)^2} .$$

In [5]:
function optimizeLambda(sigma, r, T, spot, J, strike, barrier, nSamples, optimIterations=5)
    G = Array{Array{Float64}}(undef, nSamples)
    squaredPayoff = zeros(nSamples)
    path = zeros(J + 1)
    # Store all the payoffs
    for i=1:nSamples
        G[i] = randn(J)
        bsAsset!(path, sigma, r, T, spot, J, G[i], 0)
        tmp = barrierPayoff(path, strike, barrier)
        squaredPayoff[i] = tmp * tmp
    end

    lambda = 0
    for iter=1:optimIterations
        sum0 = 0.
        sum1 = 0.
        sum2 = 0.
        for i=1:nSamples
            WT =  finalWT(G[i], T, J)
            sum0 += squaredPayoff[i] * exp(-lambda * WT)
            sum1 += WT * squaredPayoff[i] * exp(-lambda * WT)
            sum2 += WT * WT * squaredPayoff[i] * exp(-lambda * WT)
        end
        u_first = lambda * T - sum1 / sum0
        u_second = T + sum2 / sum0 - sum1 * sum1 / (sum0 * sum0)
        lambda = lambda - u_first / u_second
        println("lambda:", lambda)
    end
    return lambda
end

optimizeLambda (generic function with 2 methods)

In [12]:
lambda = optimizeLambda(0.2, 0.05, 2, 100, 24, 110, 80, 10000)
monteCarlo(0.2, 0.05, 2, 100, 24, 110, 80, 50000, lambda)


lambda:0.8605208983473653
lambda:0.9128896030171634
lambda:0.9130613530518715
lambda:0.9130613548666191
lambda:0.9130613548666164


(11.251671572121065, 0.052358932541896885)